# 🚀 Пример работы с линейной продольной моделью ракеты

## 📖 Введение

Данный notebook демонстрирует основы работы с **LinearLongitudinalMissileModel** в библиотеке TensorAeroSpace. 
Мы изучим:

- 🛠️ **Настройку параметров симуляции** для ракетной модели
- 🎯 **Создание опорных сигналов** для управления
- 🚀 **Инициализацию среды** LinearLongitudinalMissileModel
- 📊 **Выполнение шагов симуляции** и анализ результатов
- 🔍 **Исследование внутренних состояний** модели

Эта модель представляет упрощенную продольную динамику ракеты и идеально подходит для изучения основ управления летательными аппаратами.

## 🔧 Настройка рабочей среды

Переходим в корневую директорию проекта:

In [1]:
# Переход в корневую директорию проекта
print('📁 Переходим в корневую директорию...')
%cd ..

/app


## 📦 Импорт необходимых библиотек

Загружаем все необходимые модули для работы с ракетной моделью:

In [2]:
# Основные библиотеки для работы с данными и средами
import gymnasium as gym          # Стандартная библиотека для RL сред
import numpy as np              # Численные вычисления
from tqdm import tqdm           # Прогресс-бары для циклов

# Специализированные модули TensorAeroSpace
from tensoraerospace.envs import LinearLongitudinalMissileModel  # Модель ракеты
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp  # Утилиты времени
from tensoraerospace.signals.standart import unit_step  # Генератор ступенчатых сигналов
from tensoraerospace.agent.ihdp.model import IHDPAgent  # Агент IHDP

print('✅ Все библиотеки успешно загружены!')

## ⚙️ Настройка параметров симуляции

Определяем основные параметры для симуляции ракетной модели:

In [3]:
# Основные параметры симуляции
dt = 0.01  # Шаг дискретизации по времени (секунды)
print(f'🕐 Шаг дискретизации: {dt} сек')

# Генерация временного периода
tp = generate_time_period(tn=20, dt=dt)  # Временной период 20 секунд
tps = convert_tp_to_sec_tp(tp, dt=dt)    # Преобразование в секунды
number_time_steps = len(tp)             # Общее количество временных шагов

print(f'⏱️  Общее время симуляции: 20 сек')
print(f'📊 Количество временных шагов: {number_time_steps}')

# Создание опорного сигнала (ступенчатый сигнал)
reference_signals = np.reshape(
    unit_step(degree=5, tp=tp, time_step=10, output_rad=True), 
    [1, -1]
)

print(f'🎯 Опорный сигнал создан:')
print(f'   • Амплитуда: 5 градусов')
print(f'   • Время активации: 1.0 сек (шаг {10})')
print(f'   • Форма сигнала: {reference_signals.shape}')

## 🚀 Создание среды ракетной модели

Инициализируем среду LinearLongitudinalMissileModel с заданными параметрами:

In [4]:
# Создание среды LinearLongitudinalMissileModel
print('🏗️  Создаем среду ракетной модели...')

env = gym.make('LinearLongitudinalMissileModel-v0',
               number_time_steps=number_time_steps,    # Количество временных шагов
               initial_state=[[0],[0],[0],[0]],        # Начальное состояние [x, x_dot, theta, theta_dot]
               reference_signal=reference_signals)     # Опорный сигнал

print('✅ Среда успешно создана!')
print('🔄 Выполняем сброс среды...')

# Сброс среды и получение начального состояния
initial_observation, info = env.reset()

print('📊 Начальное состояние ракеты:')
print(f'   • Позиция (x): {initial_observation[0][0]:.6f} м')
print(f'   • Скорость (x_dot): {initial_observation[1][0]:.6f} м/с')
print(f'   • Угол тангажа (theta): {initial_observation[2][0]:.6f} рад')
print(f'   • Угловая скорость (theta_dot): {initial_observation[3][0]:.6f} рад/с')

initial_observation, info

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


(array([[0.],
        [0.],
        [0.],
        [0.]], dtype=float32),
 {})

## 🎮 Выполнение шага симуляции

Применяем управляющее воздействие и наблюдаем реакцию ракетной системы:

In [6]:
# Применение управляющего воздействия
control_input = np.array([[1]])  # Управляющий сигнал (тяга)

print('🎯 Применяем управляющее воздействие...')
print(f'   • Управляющий сигнал: {control_input[0][0]} (единичная тяга)')

# Выполнение одного шага симуляции
observation, reward, terminated, truncated, info = env.step(control_input)

print('✅ Шаг симуляции выполнен!')
print(f'🏁 Статус завершения: terminated={terminated}, truncated={truncated}')

## 🔍 Анализ внутренних данных модели

Исследуем внутренние состояния и данные ракетной модели:

In [11]:
# Анализ истории управляющих воздействий
input_history = env.unwrapped.model.store_input

print('📈 История управляющих воздействий:')
print(f'   • Форма массива: {input_history.shape}')
print(f'   • Первые 5 значений: {input_history[0][:5]}')
print(f'   • Последние 5 значений: {input_history[0][-5:]}')

# Подсчет ненулевых управляющих воздействий
non_zero_inputs = np.count_nonzero(input_history)
print(f'🎯 Количество ненулевых управляющих воздействий: {non_zero_inputs}')

input_history

array([[1., 1., 0., ..., 0., 0., 0.]])

### 🎯 Анализ опорного сигнала

Исследуем текущее значение опорного сигнала:

In [10]:
# Анализ текущего значения опорного сигнала
current_reference = env.unwrapped.reference_signal[0][1]

print('🎯 Анализ опорного сигнала:')
print(f'   • Текущее значение (шаг 1): {current_reference:.6f} рад')
print(f'   • Значение в градусах: {np.degrees(current_reference):.2f}°')

# Показать несколько значений опорного сигнала
ref_signal = env.unwrapped.reference_signal[0]
print(f'📊 Форма опорного сигнала: {ref_signal.shape}')
print(f'   • Первые 5 значений: {ref_signal[:5]}')
print(f'   • Значения 8-12 (около активации): {ref_signal[8:13]}')

current_reference

0.0

### 🏆 Анализ награды

Оценим качество управления через полученную награду:

In [9]:
# Анализ полученной награды
current_reward = reward[0] if isinstance(reward, np.ndarray) else reward

print('🏆 Анализ награды:')
print(f'   • Текущая награда: {current_reward:.6f}')

# Интерпретация награды
if current_reward > 0:
    print('   ✅ Положительная награда - хорошее управление!')
    print('   📈 Система движется в правильном направлении')
elif current_reward == 0:
    print('   ⚖️  Нейтральная награда - система в равновесии')
else:
    print('   ⚠️  Отрицательная награда - требуется коррекция')

print('
📊 Интерпретация награды:')
print('   • Награда отражает качество отслеживания опорного сигнала')
print('   • Высокие значения указывают на точное следование траектории')
print('   • Отрицательные значения сигнализируют об ошибках управления')

reward

array([0.19481281])

---

## 🎓 Заключение

В этом примере мы успешно продемонстрировали работу с **LinearLongitudinalMissileModel** в TensorAeroSpace! 🚀

### ✅ Что мы изучили:

- 🛠️ **Настройку параметров симуляции** для ракетной модели
- 🎯 **Создание опорных сигналов** с помощью unit_step
- 🚀 **Инициализацию среды** LinearLongitudinalMissileModel
- 🎮 **Выполнение шагов симуляции** с управляющими воздействиями
- 🔍 **Анализ внутренних состояний** модели и истории управления
- 📊 **Интерпретацию наград** и оценку качества управления

### 🔄 Следующие шаги для изучения:

- 🎮 **Экспериментируйте с различными управляющими воздействиями**
- 📈 **Создайте более сложные опорные сигналы** (синусоидальные, рамповые)
- 🧠 **Реализуйте контроллеры** (PID, MPC, или RL-агенты)
- 📊 **Визуализируйте траектории** и динамику системы
- 🔬 **Изучите влияние параметров** на поведение ракеты

### 📚 Полезные ресурсы:

- [Документация TensorAeroSpace](https://tensoraerospace.readthedocs.io/)
- [Примеры контроллеров](https://github.com/TensorAeroSpace/TensorAeroSpace/tree/main/examples)
- [Теория управления ракетами](https://en.wikipedia.org/wiki/Missile_guidance)

**Удачи в изучении ракетных систем и управления! 🚀🎯**